# Building Agents with Semantic Kernel

This notebook demonstrates how to create and orchestrate agents workflows using Semantic Kernel.

**Objectives:**
- Set up and configure agents in Semantic Kernel using ChatCompletionAgent.
- Add tools (plugins) to agents.

## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [1]:
// Import Semantic Kernel and required packages
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.55.0 Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [2]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3**: Instantiate the Kernel and Chat Clients

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models and agents.

In [3]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("TEXT_MODEL_NAME");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client); 

# Bike Inventory Agent

This section demonstrates how to define and use a bike inventory agent that answers questions about bike availability and features at Contoso Bike Store.

**Step 1:** Create the plugin to simulate the bike inventory.

In [4]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.Collections.Generic;

public class BikeInventoryPlugin
{
    private readonly Dictionary<string, string> _inventory = new()
    {
        { "Contoso Explorer", "A rugged mountain bike suitable for off-road and trail riding." },
        { "Contoso CityLite", "A lightweight city bike perfect for commuting and urban rides." },
        { "Contoso TrailBlazer", "A versatile hybrid bike for both city and light trail use." },
        { "Contoso Speedster", "A fast road bike designed for speed and long-distance rides." }
    };

    [KernelFunction]
    public string CheckBikeAvailability(string model)
    {
        if (_inventory.ContainsKey(model))
            return $"Yes, the {model} is available. {model} details: {_inventory[model]}";
        return $"Sorry, the {model} is currently out of stock.";
    }

    [KernelFunction]
    public string ListAvailableBikes()
    {
        return "Available bikes: " + string.Join(", ", _inventory.Keys);
    }
}

**Step 2**: Create the bike inventory agent and register it with the Kernel

In [5]:

var bikeInventoryAgentName = "BikeInventoryAgent";
var bikeInventoryAgentInstructions = @"
You are a helpful assistant for Contoso Bike Store. Use the BikeInventoryPlugin to answer questions about bike availability and features.
If the user asks for a bike that is not in stock, inform them politely.
If the user asks for a list of available bikes, provide the list.
If the user asks for other information, politely decline and suggest they ask about bikes in the store.
Always respond in a single turn.
";

// Get the chat completion service
Kernel kernel = builder.Build();
// Register the plugin with the kernel
var bikeInventoryPlugin = new BikeInventoryPlugin();
kernel.Plugins.AddFromObject(bikeInventoryPlugin, "BikeInventoryPlugin");

// Define the bike inventory agent
ChatCompletionAgent bikeInventoryAgent = new()
{
    Name = bikeInventoryAgentName,
    Instructions = bikeInventoryAgentInstructions,
    Kernel = kernel,
};


**Step 3**: Create a helper function to call the agent and handle the response.

In [28]:

var invokeAgentAsync = async (string input,AgentThread thread) =>
{
     IAsyncEnumerable<AgentResponseItem<ChatMessageContent>> responses = 
     bikeInventoryAgent.InvokeAsync(input, thread);

    await foreach (var response in responses)
    {
        // AuthorName usage is suppressed for evaluation purposes
        Console.WriteLine(response.Message.Content);
        thread = response.Thread; // Continue with same thread
    }
};

**Step 4** Now let us try invoking the agent with different queries.


**Query 1:** Check if the Contoso Speedster is available.

In [31]:

AgentThread thread = new ChatHistoryAgentThread();
await invokeAgentAsync("Is the Contoso Speedster available?", thread);

Let me check on that for you.

The Contoso Speedster is in stock! Let me know if you'd like more details about it.


**Query 2:** List all available bikes.

In [33]:

AgentThread thread = new ChatHistoryAgentThread();
await invokeAgentAsync("Can you list all available bikes?", thread);

Sure! Let me get the list of available bikes for you.  

**Available Bikes:**  
1. Contoso Roadster 200 - Road bike - $599  
2. Contoso Trailblazer 300 - Mountain bike - $749  
3. Contoso Cruiser Classic - Cruiser bike - $499  
4. Contoso Speedster 500 - Hybrid bike - $899  

Let me know if you'd like details about any specific bike!


**Query 3:** Ask about car accessories (an unsupported query).

In [34]:
AgentThread thread = new ChatHistoryAgentThread();
await invokeAgentAsync("Do you sell car accessories?", thread);

I'm here to assist with information about bikes in the Contoso Bike Store. Feel free to ask about bike availability or features! We don't provide information about car accessories.
